In [43]:
import os

import pandas as pd
import numpy as np

import tensorflow as tf


import matplotlib.pyplot as plt
import seaborn as sns


##### Declaração das constantes utilizadas no código

In [98]:
DATASET_PATH = "dataset"
CONTEXT_TEXT_PATH = os.path.join(DATASET_PATH,"text_data")

S08_QA = ["S08_question_answer_pairs.txt"]
S09_QA = ["S09_question_answer_pairs.txt"]
S10_QA = ["S10_question_answer_pairs.txt"]
ALL_QA = S08_QA + S09_QA + S10_QA

PUNCTUATIONS_TO_REMOVE = [".", "!", "?"]
BAD_ANSWER_TO_REMOVE = ["blah","blah blah blah","(What?)"]


In [99]:
# carrega conteúdo dos arquivos com o texto utilizado para responder as perguntas
def get_context_files():
    context_file_map = {}
    for f in os.listdir(CONTEXT_TEXT_PATH):
        file_full_path = os.path.join(CONTEXT_TEXT_PATH, f)
        if os.path.isfile(file_full_path):
            with open(file_full_path, 'r', encoding="utf-8") as f_contexto:
                contexto = f_contexto.read().replace('\n', '')
            chave = f.replace(".txt.clean","")
            context_file_map[chave] = contexto
    return context_file_map
    
#context_files = get_context_files()
#  lista_tamanhos = []
#  for cf in context_files:
#       lista_tamanhos.append(len(context_files[cf].split()))
#   print(lista_tamanhos[:5])


In [ ]:
def load_qa_pairs(qa_source=ALL_QA):
    df_list = []
    for qa_file in qa_source:
        df= pd.read_csv(os.path.join(DATASET_PATH,qa_file),sep='\t')
        df_list.append(df)
    df_concat = pd.concat(df_list)
    return df_concat

In [ ]:
df_inicial = load_qa_pairs(qa_source=S08_QA)

##### Análise inicial, somente com as primeiras 10 linhas:
- Case diversos - deve ser tratado.
- Algumas respostas tem pontuação - deve ser tratado.
- Algumas perguntas não incluem o contexto - Did his mother die of pneumonia?. Validar se quando o title não aparecer no texto ele deve ser incluído.

In [ ]:
df_inicial.head(10)

##### Missing Values
- Retirar perguntas nulas
- Retirar respostas nulas

In [ ]:
print(df_inicial.isna().sum())
print(df_inicial.shape)

In [ ]:
#verificar se existe alguma inconsistencia como resposta para pergunta nual
df_inicial[(df_inicial.Question.isna()) &
           (df_inicial.Answer.notna())]

In [ ]:
df_inicial[df_inicial.ArticleFile.isna()]

In [ ]:
# retirando missing values
df_preparado = df_inicial[(df_inicial.Question.notna()) &
                          (df_inicial.Answer.notna())]
print(df_preparado.shape)

In [ ]:
df_preparado.head(50)

In [ ]:
df_preparado.Answer.value_counts()

#### Preparação dos dados

Questões para tratamento
- Lowercase
- Pontuação da resposta, ultimo caracter com . ou ! ou ? devem ser retirados.
- Pontuação da pergunta, devemos tirar a interrogação?
- Retirar perguntas duplicadas: Respostas null já forma retiradas no pandas. Escolhe sempre a primeira.
- Remover respostas ruins: blah, (What)?, blah blah blah

In [90]:
def normalize_qa(vet_qa):
    vet_n_resp = []
    for qa in vet_qa:
        pergunta, resposta = qa[0], qa[1]
        pergunta = pergunta.lower()
        
        if (resposta[-1] in PUNCTUATIONS_TO_REMOVE):
            resposta = resposta[:-1]
        vet_n_resp.append([pergunta, resposta])
    return np.array(vet_n_resp, dtype=object)


In [93]:
def get_pergunta_igual_from_position(pos, vet, pergunta):
    vet_resp = []
    for index in range(pos, len(vet)):
        if vet[index][0] == pergunta:
            vet_resp.append(vet[index])
    return vet_resp
        

In [94]:
def remove_duplication(vet_qa):
    pergunta_set = set()
    vet_qa_clean = []
    
    for pos,qa in enumerate(vet_qa):
        pergunta, resposta = qa[0], qa[1]
        if pergunta in pergunta_set:
            continue
        #perguntas_duplicadas = get_pergunta_igual_from_position(pos+1, vet_qa, pergunta)
        pergunta_set.add(pergunta)
        vet_qa_clean.append(qa)
    return np.array(vet_qa_clean, dtype=object)            
    

In [ ]:
def remove_bad_answer(vet_qa):
    vet_qa_resp = []
    for qa in vet_qa:
        pergunta, resposta = qa[0], qa[1]
        if pergunta in BAD
    
    

---

In [97]:
vet_QA = df_preparado[["Question", "Answer"]].to_numpy()
vet_QA = normalize_qa(vet_QA)
print('Antes remoção duplicadas', vet_QA.shape)
vet_QA = remove_bad_answer(vet_QA)
print('Após remoção duplicadas', vet_QA.shape)
vet_QA = remove_duplication(vet_QA)
print('Após remoção duplicadas', vet_QA.shape)

Antes remoção duplicadas (1475, 2)
Após remoção duplicadas (916, 2)


In [96]:
vet_QA[:50]

array([['was abraham lincoln the sixteenth president of the united states?',
        'yes'],
       ['did lincoln sign the national banking act of 1863?', 'yes'],
       ['did his mother die of pneumonia?', 'no'],
       ["how many long was lincoln's formal education?", '18 months'],
       ['when did lincoln begin his political career?', '1832'],
       ['what did the legal tender act of 1862 establish?',
        'the United States Note, the first paper currency in United States history'],
       ['who suggested lincoln grow a beard?',
        '11-year-old Grace Bedell'],
       ['when did the gettysburg address argue that america was born?',
        '1776'],
       ['did lincoln beat john c. breckinridge in the 1860 election?',
        'yes'],
       ['was abraham lincoln the first president of the united states?',
        'No'],
       ['did lincoln start his political career in 1832?', 'Yes'],
       ['did lincoln ever represent alton & sangamon railroad?', 'Yes'],
       ['which c